In [ ]:
import json
import pandas as pd
from sklearn.model_selection import train_test_split

def filter_by_freq(df: pd.DataFrame, column: str, min_freq: int) -> pd.DataFrame:
    freq = df[column].value_counts()
    frequent_values = freq[freq > min_freq].index
    return df[df[column].isin(frequent_values)]
def create_id_map(sampled_ids):
    sampled_ids = sorted(sampled_ids)
    id_map = {x:i for i,x in enumerate(sampled_ids)}
    return id_map
def print_sample_summary(df, user_col_name='userId', item_col_name='movieId'):
    n_unique_users = len(set(df[user_col_name]))
    n_unique_items = len(set(df[item_col_name]))
    n_ratings = len(df)
    print(n_ratings, " ratings, ", n_unique_users, " users, ", n_unique_items, " items")
    print("Sparsity: ", n_ratings/(n_unique_users*n_unique_items))

In [ ]:
lines = []
for line in open('../datasets/reviews.clean.json').readlines():
    lines.append(eval(line))
df = pd.DataFrame(lines)
df = df.drop(['reviewerName', 'reviewText', 'categories', 'reviewTime'], axis=1)
df.to_csv('../datasets/gl_df.csv')

In [ ]:
df = pd.read_csv('../datasets/gl_df.csv')
print_sample_summary(df, 'gPlusUserId', 'gPlusPlaceId')

In [ ]:
filtered_df = filter_by_freq(df, 'gPlusUserId', 150)
print_sample_summary(filtered_df, 'gPlusUserId', 'gPlusPlaceId')
filtered_df.to_csv('../datasets/filtered_gl_df')

In [ ]:
import numpy as np
sampled_uids = list(set(filtered_df['gPlusUserId']))
sampled_uids = np.random.choice(sampled_uids, 1500, replace=False)
sampled_df = filtered_df[filtered_df['gPlusUserId'].isin(sampled_uids)]
sampled_mids = list(set(sampled_df['gPlusPlaceId']))
print_sample_summary(sampled_df, 'gPlusUserId', 'gPlusPlaceId')
uid_map = create_id_map(sampled_uids)
mid_map = create_id_map(sampled_mids)
sampled_df['uid'] = sampled_df['gPlusUserId'].map(uid_map)
sampled_df['mid'] = sampled_df['gPlusPlaceId'].map(mid_map)
print_sample_summary(sampled_df, 'gPlusUserId', 'gPlusPlaceId')

In [ ]:
sampled_df.to_csv('../datasets/gl/u.data')
n_runs = 5
for i in range(n_runs):
    X = sampled_df.copy()
    X = X.sample(frac=1)
    uids = X.copy().pop('uid').to_frame()
    X_train, X_test, y_train, y_test = train_test_split(X, uids,stratify=uids, test_size=0.2)
    X_train['uid'] = y_train.values.squeeze()
    X_test['uid'] = y_test.values.squeeze()
    X_train.to_csv('../datasets/gl/u' + str(i) + '.base')
    X_test.to_csv('../datasets/gl/u' + str(i) + '.test')

In [ ]:
import numpy as np
sampled_uids = list(set(filtered_df['gPlusUserId']))
sampled_uids = np.random.choice(sampled_uids, 500, replace=False)
sampled_df = filtered_df[filtered_df['gPlusUserId'].isin(sampled_uids)]
sampled_mids = list(set(sampled_df['gPlusPlaceId']))

uid_map = create_id_map(sampled_uids)
mid_map = create_id_map(sampled_mids)
sampled_df['uid'] = sampled_df['gPlusUserId'].map(uid_map)
sampled_df['mid'] = sampled_df['gPlusPlaceId'].map(mid_map)
print_sample_summary(sampled_df, 'gPlusUserId', 'gPlusPlaceId')

In [ ]:
sampled_df.to_csv('../datasets/gl-tiny/u.data')
n_runs = 5
for i in range(n_runs):
    X = sampled_df.copy()
    X = X.sample(frac=1)
    uids = X.copy().pop('uid').to_frame()
    X_train, X_test, y_train, y_test = train_test_split(X, uids,stratify=uids, test_size=0.2)
    X_train['uid'] = y_train.values.squeeze()
    X_test['uid'] = y_test.values.squeeze()
    X_train.to_csv('../datasets/gl-tiny/u' + str(i) + '.base')
    X_test.to_csv('../datasets/gl-tiny/u' + str(i) + '.test')